In [1]:
#Initial Imports
import json
import requests
import pandas as pd
from collections import Counter

In [35]:
#Setting batch 1 post range
id_list1 = []
for i in range(274307863, 264307863, -1000000):
    id_list1.append(i)

In [36]:
#Setting batch 2 post range
id_list2 = []
for i in range(264307863, 254307863, -1000000):
    id_list2.append(i)

In [37]:
#Setting batch 3 post range
id_list3 = []
for i in range(254307863, 244307863, -1000000):
    id_list3.append(i)

In [91]:
#Function to grab posts from stocktwits API
def finding_posts(id_list, symbol):
    
    combined = []
    
    for ids in id_list:
        
        max = ids
        symbol_id = symbol
        url = "https://api.stocktwits.com/api/2/"
        headers = {'Content-Type': 'application/json'}
    
        form_url = url + 'streams/symbol/' + symbol_id + '.json'
        params = {'max': max}
        r = requests.get(form_url, headers=headers, params=params)
        raw_json = r.json()
        tsla_tweets = raw_json['messages']
        
        for tweets in range(0,len(tsla_tweets)):
            data = []
            data.append(tsla_tweets[tweets]['body'])
            data.append(tsla_tweets[tweets]['created_at'])
            data.append(tsla_tweets[tweets]['user']['like_count'])
            sentiment = tsla_tweets[tweets]['entities']
            sentiment_check = sentiment.get('sentiment', 0)
            data.append(sentiment_check)
            
            combined.append(data)
    
    return combined

In [125]:
#Getting stocktwits tweets
st_batch_1 = finding_posts(id_list1, "PLUG")
st_batch_2 = finding_posts(id_list2, "PLUG")
st_batch_3 = finding_posts(id_list3, "PLUG")

In [136]:
#Saving tweets to a df
st_posts_1 = pd.DataFrame(st_batch_1, columns=['Text', 'Created', 'Likes', 'Sentiment'])
st_posts_2 = pd.DataFrame(st_batch_2, columns=['Text', 'Created', 'Likes', 'Sentiment'])
st_posts_3 = pd.DataFrame(st_batch_3, columns=['Text', 'Created', 'Likes', 'Sentiment'])

In [182]:
#Joining batches together into master df
st_joined = pd.concat([st_posts_1, st_posts_2, st_posts_3])
st_joined.head()

In [173]:
#Formatting 'Sentiment' column
st_joined.Sentiment = st_joined.Sentiment.astype(str) 
st_joined['Sentiment'] = st_joined['Sentiment'].astype(str).replace("b", "")

st_joined['Sentiment'] = st_joined['Sentiment'].str.replace('{','')
st_joined['Sentiment'] = st_joined['Sentiment'].str.replace('}','')
st_joined['Sentiment'] = st_joined['Sentiment'].str.replace("'",'')
st_joined['Sentiment'] = st_joined['Sentiment'].str.replace("basic",'')
st_joined['Sentiment'] = st_joined['Sentiment'].str.replace(": ",'')

In [201]:
#Removing timestamp from 'Created' column
st_joined['Created'] = pd.to_datetime(st_joined['Created'])
st_joined['Created'] = st_joined['Created'].dt.date

In [141]:
#Saving df as CSV
st_joined.to_csv(r'Data/tsla_tweets.csv', header=True, index=False)